# Super-G Sprite Testing

###### V. Hunter Adams

In [22]:
import serial
import numpy
from matplotlib import pyplot as plt
from IPython.display import clear_output
from IPython.display import Image
from IPython.display import Latex
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import HTML 
from IPython.display import HTML
from ipywidgets import *
import plotly.plotly as py
import plotly
import plotly.graph_objs as go
import pandas as pd
import plotly.tools as tls
import urllib2
tls.set_credentials_file(username='v.h.adams', api_key='werEw55EhyYuY92HZtTt')

## Experimental Setup

Header pins were soldered onto each of the 40 Sprites pictured below so that each could be snapped into a breadboard. The breadboard connects the Sprite to an FTDI, which is in turn connected to the computer via USB. Each Sprite was loaded with a variation of the software shown below (the only variation being the name of each Sprite, a number between 0 and 40). 200 datapoints from each sensor were collected from each Sprite. Data will be gathered in precisely the same manner after g-testing to characterize any degradatation that may have taken place.

#### Hardware

<img src="Sprites.png">

<img src="Breadboard.png">

#### Software

```C

#include <Arduino.h>
#include <Wire.h>
#include "HardwareSerial.h"
#include <LSM9DS1.h>

int back_sunsensor=13;
int front_sunsensor=8;

int sprite_name=40; // Different for each Sprite

// the setup function runs once when you press reset or power the board
void setup() {
    Wire.begin();
    smeGyroscope.begin();
    smeAccelerometer.begin();
    smeMagnetic.begin();
    Serial.begin(9600);
}

void printAxis(int x, int y, int z, int xa, int ya, int za, int xm, int ym, int zm, int front, int back) {
    Serial.print(sprite_name, DEC);
    Serial.print(",");
    Serial.print(x, DEC);
    Serial.print(",");
    Serial.print(y, DEC);
    Serial.print(",");
    Serial.print(z, DEC);
    Serial.print(",");
    Serial.print(xa, DEC);
    Serial.print(",");
    Serial.print(ya, DEC);
    Serial.print(",");
    Serial.print(za, DEC);
    Serial.print(",");
    Serial.print(xm, DEC);
    Serial.print(",");
    Serial.print(ym, DEC);
    Serial.print(",");
    Serial.print(zm, DEC);
    Serial.print(",");
    Serial.print(front, DEC);
    Serial.print(",");
    Serial.println(back, DEC);
}

// the loop function runs over and over again forever
void loop() {

    int x = 0;
    int y = 0;
    int z = 0;
    int xa = 0;
    int ya = 0;
    int za = 0;
    int xm = 0;
    int ym = 0;
    int zm = 0;
    int front = 0;
    int back = 0;

    x = smeGyroscope.readX();
    y = smeGyroscope.readY();
    z = smeGyroscope.readZ();
    xa = smeAccelerometer.readX();
    ya = smeAccelerometer.readY();
    za = smeAccelerometer.readZ();
    xm = smeMagnetic.readX();
    ym = smeMagnetic.readY();
    zm = smeMagnetic.readZ();
    front = analogRead(front_sunsensor);
    back = analogRead(back_sunsensor);

//    Serial.print("Gyroscope     [mdps]    :");
    printAxis(x, y, z, xa, ya, za, xm, ym, zm, front, back);

    digitalWrite(LED, LOW);
    delay(10);

    digitalWrite(LED, HIGH);    // turn the LED on
    delay(10);            // wait for a second

}
```

## Data Acquisition

Function that writes serial data from Sprite to a file.

In [48]:
ser = serial.Serial('/dev/cu.usbserial-DN01AWD9', 9600)
filename="./Posterior_Data/sprite_5.txt"

datafile=open(filename, 'a')
count = 0;
while count < 200:
    datafile=open(filename, 'a')
    data = ser.readline().rstrip()
    datafile.write(data)
    datafile.write('\n')
    print data
    count += 1
    datafile.close()
datafile.close()
ser.close()
clear_output()

## Helper Functions and Global Variables

Indices representing media of impact:

In [ ]:
air      = numpy.array([9,10,15,16,22,21])-1
beads    = numpy.array([11,12,17,18,24,23])-1
wax      = numpy.array([1,2,3,4,7,8,5,6])-1
wcs8     = numpy.array([13,14,20,19,26,25])-1
stf      = numpy.array([39, 40])-1
regolith = numpy.array([28,27,29,30,31,32,33,34,35,36,37,38])-1

Create a function that builds a histogram object:

In [49]:
def createHisto(data, label):
    return go.Histogram(
                x=data,
                histnorm='probability',
                name=label,
                autobinx=True,
                opacity=0.75
            )

Global variables manipulated during histogram construction, and general layout object:

In [50]:
total=40
data = []
count = 0
means = []
stds = []
zmeandata = []

layout = go.Layout(
    barmode='stack',
    xaxis=dict(
        title='Measured Value'
    ),
    yaxis=dict(
        title='Count'
    ),
    bargap=0.1,
    bargroupgap=0.1
)

Function for extracting data in specified line location from all files in directory:

In [51]:
def gatherData(index):
    global data
    global count
    global means
    global stds
    global zmeandata
    count = 0
    data, means, stds, zmeandata = [], [], [], []
    for filename in os.listdir('./Posterior_Data'):
        x = []
        count += 1
        with open('./Posterior_Data/'+filename, "rt") as f:
            for line in f:
                try:
                    if len(line.split(',')) == 12:
                        x.extend([int(line.split(',')[index])])
                    else:
                        continue
                except:
                    continue
        data.extend([createHisto(x, 'Sprite '+str(count))])
        means.extend([numpy.mean(x)])
        stds.extend([numpy.std(x)])
        temp = list(numpy.array(x) - numpy.mean(x))
        zmeandata.extend([createHisto(temp, 'Sprite '+str(count))])
#         print 'Sprite Number '+str(count)
#         print '------------------------------'
#         print 'Mean: '+str(numpy.mean(x))
#         print 'Std: '+str(numpy.std(x)) + '\n\n'
        
    print 'Mean of means: '+str(numpy.mean(means))
    print 'Std of means: '+str(numpy.std(means))
    print 'Mean of Std\'s: '+str(numpy.mean(stds))
    print 'Std of std\'s: '+str(numpy.std(stds))
    print 'Number functioning: '+str(count)
    print 'Number dead: '+str(total-count)

## Gyroscope

#### X-Measurements

In [25]:
gatherData(1)

Mean of means: 504.565848891
Std of means: 698.958181588
Mean of Std's: 38.0044646605
Std of std's: 15.8211558728
Number functioning: 40
Number dead: 0


In [26]:
layout.title='Measured Gyroscope x-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_gx')

In [52]:
gatherData(1)

Mean of means: 147.503366377
Std of means: 1059.17380964
Mean of Std's: 42.7656773189
Std of std's: 24.6995386903
Number functioning: 40
Number dead: 0


In [53]:
layout.title='Measured Gyroscope x-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_gx_posterior')

In [27]:
layout.title = 'Unbiased Gyroscope x-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_gx_zm')

In [28]:
layout.title = 'Unbiased Gyroscope x-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_gx_zm_posterior')

#### Y-Measurements

In [28]:
gatherData(2)

Mean of means: 45.7881632722
Std of means: 956.885750505
Mean of Std's: 45.4013344332
Std of std's: 18.218526426
Number functioning: 40
Number dead: 0


In [29]:
layout.title='Measured Gyroscope y-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_gy')

In [30]:
layout.title = 'Unbiased Gyroscope y-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_gy_zm')

#### Z-Measurements

In [31]:
gatherData(3)

Mean of means: 168.635759359
Std of means: 1123.81009974
Mean of Std's: 33.3752903088
Std of std's: 10.5315361453
Number functioning: 40
Number dead: 0


In [32]:
layout.title='Measured Gyroscope z-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_gz')

In [33]:
layout.title = 'Unbiased Gyroscope z-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_gz_zm')

## Accelerometer

#### X-Measurements

In [34]:
gatherData(4)

Mean of means: 400.077493554
Std of means: 22.4274809805
Mean of Std's: 1.04917265181
Std of std's: 0.120670087416
Number functioning: 40
Number dead: 0


In [35]:
layout.title='Measured Accelerometer x-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_ax')

In [36]:
layout.title = 'Unbiased Accelerometer x-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_ax_zm')

#### Y-Measurements

In [37]:
gatherData(5)

Mean of means: -16.3370312992
Std of means: 12.0381621181
Mean of Std's: 0.648596517151
Std of std's: 0.0604201144231
Number functioning: 40
Number dead: 0


In [38]:
layout.title='Measured Accelerometer y-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_ay')

In [39]:
layout.title = 'Unbiased Accelerometer y-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_ay_zm')

#### Z-Measurements

In [40]:
gatherData(6)

Mean of means: -927.918580783
Std of means: 18.5574060912
Mean of Std's: 2.2593895557
Std of std's: 0.497775163618
Number functioning: 40
Number dead: 0


In [41]:
layout.title='Measured Accelerometer z-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_az')

In [42]:
layout.title = 'Unbiased Accelerometer z-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_az_zm')

## Magnetometer

#### X-Measurements

In [43]:
gatherData(7)

Mean of means: 716.096493642
Std of means: 136.386271579
Mean of Std's: 10.8768854137
Std of std's: 0.66877103902
Number functioning: 40
Number dead: 0


In [44]:
layout.title='Measured Magnetometer x-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_mx')

In [45]:
layout.title = 'Unbiased Accelerometer x-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_ax_zm')

#### Y-Measurements

In [46]:
gatherData(8)

Mean of means: 49.2828393863
Std of means: 134.720174355
Mean of Std's: 7.22869954899
Std of std's: 0.460715779577
Number functioning: 40
Number dead: 0


In [47]:
layout.title='Measured Magnetometer y-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_my')

In [48]:
layout.title = 'Unbiased Accelerometer y-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_ay_zm')

#### Z-Measurements

In [49]:
gatherData(9)

Mean of means: 180.599937903
Std of means: 71.0376932958
Mean of Std's: 6.15021658588
Std of std's: 0.371342388016
Number functioning: 40
Number dead: 0


In [50]:
layout.title='Measured Magnetometer z-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_mz')

In [51]:
layout.title = 'Unbiased Accelerometer z-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_az_zm')

## Phototransistors

#### Rear

In [52]:
gatherData(10)

Mean of means: 165.347115432
Std of means: 25.3976280936
Mean of Std's: 4.56545452219
Std of std's: 2.28221604837
Number functioning: 40
Number dead: 0


In [53]:
layout.title='Measured Rear Light Sensor'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_lr')

In [54]:
layout.title = 'Unbiased Rear Light Sensor'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_lr_zm')

#### Front

In [55]:
gatherData(11)

Mean of means: 2.71150939673
Std of means: 0.980065188618
Mean of Std's: 2.62227648698
Std of std's: 0.323748744816
Number functioning: 40
Number dead: 0


In [56]:
layout.title='Measured Front Light Sensor'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_lf')

In [57]:
layout.title = 'Unbiased Front Light Sensor'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_lf_zm')